In [1]:
import os
from os import listdir
from os.path import isfile,join
from io import StringIO
import pandas as pd
from collections import Counter
import en_core_web_sm
nlp=en_core_web_sm.load()
from spacy.matcher import PhraseMatcher
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO 
import re
import pandas
from pandas import ExcelWriter
import datetime
from tkinter import filedialog
from tkinter import Button, Tk, HORIZONTAL
from tkinter.ttk import Progressbar
import time
import threading

In [2]:
#optional parameter PAGES can restrict which pages to process
def convert_pdf_to_txt(path, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)
    
    #Instantiate the PDFminer objects
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)
    
    infile = open(path, 'rb') #open the file for read in binary mode
    
    for page in PDFPage.get_pages(infile, pagenums):
        #iterate with the pdf interpreter through the pdf pages
        interpreter.process_page(page)
    
    #Close the files and converters
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()
    text=str(text)
    text=text.replace("\\n"," ")
    text=text.replace("\n"," ")
    text=text.replace(" \x0c "," ")
    text=text.lower()
    return text   #return the text as str

def removeques(text):
    text=re.sub(r"\bq1. why should you be hired for this internship?\b"," " ,text)
    text=re.sub(r"\bq4. which is your strongest open source tool among sas, r, tableau, python?\b"," " ,text)
    text=re.sub(r"\bq3. how will you rate yourself on sql knowledge?\b"," " ,text)
    return text
def createProfile(file,onto):
    text=convert_pdf_to_txt(file)
    text=removeques(text)
    #print(text)
    keyword_dict=pd.read_csv(str(onto))
    
    time_word=[nlp(text) for text in keyword_dict["TimeSeries"].dropna(axis=0)]
    nlp_word=[nlp(text) for text in keyword_dict["NLP"].dropna(axis=0)]
    ml_word=[nlp(text) for text in keyword_dict["Machine Learning"].dropna(axis=0)]
    python_word=[nlp(text) for text in keyword_dict["Python"].dropna(axis=0)]
    r_word=[nlp(text) for text in keyword_dict["R"].dropna(axis=0)]
    dl_word=[nlp(text) for text in keyword_dict["Deep Learning"].dropna(axis=0)]
    BI_word=[nlp(text) for text in keyword_dict["BI Tools"].dropna(axis=0)]
    dataeng_word=[nlp(text) for text in keyword_dict["Data Engineering"].dropna(axis=0)]
    big_word=[nlp(text) for text in keyword_dict["BigData"].dropna(axis=0)]
    
    matcher=PhraseMatcher(nlp.vocab)
    matcher.add('TimeSeries',None,*time_word)
    matcher.add('BI_Tools',None,*BI_word)
    matcher.add('R',None,*r_word)
    matcher.add('NLP',None,*nlp_word)
    matcher.add('Bigdata',None,*big_word)
    matcher.add('Machine_learning',None,*ml_word)
    matcher.add('Python',None,*python_word)
    matcher.add('DeepLearning',None,*dl_word)
    matcher.add('DataEngineering',None,*dataeng_word)
    doc=nlp(text)
    
    d=[]
    matches=matcher(doc)
    for match_id,start,end in matches:
        rule_id=nlp.vocab.strings[match_id]
        span=doc[start:end]
        d.append((rule_id,span.text))
    keywords="\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(d).items())
    
    df=pd.read_csv(StringIO(keywords),names=['Keywords_list'])
    
    df1=pd.DataFrame(df.Keywords_list.str.split(' ',1).tolist(),columns=['Subject','Keyword'])
    df2=pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns=['Keyword','Count'])
    df3=pd.concat([df1['Subject'],df2['Keyword'],df2['Count']],axis=1)
    df3['Count']=df3['Count'].apply(lambda x:x.rstrip(")"))
    #print(df)
    #print(df1)
    #print(df2)
    #print(df3)
    base=os.path.basename(file)
    filename=os.path.splitext(base)[0]
    name = filename.split('_')
    name2 = name[0]
    name2 = name2.lower()
    ## converting str to dataframe
    name3 = pd.read_csv(StringIO(name2),names = ['Candidate Name'])
    
    dataf = pd.concat([name3['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
    dataf['Candidate Name'].fillna(dataf['Candidate Name'].iloc[0], inplace = True)
    #return text
    return(dataf)
    #return "Generating candidate skill profile"
def get_report(folder,ontol):
   
    mypath=folder
    onlyfiles=[os.path.join(mypath,f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,f))]
    #print(onlyfiles)
    onto=str(ontol)
  
    
    #print(onto)
    #print(len(onlyfiles))
    currentDT = datetime.datetime.now()
    currentDT=currentDT.strftime("%Y%m%d%H%M%S")
    final_database=pd.DataFrame()
    i = 0 
    print("--------------------")
    while i < len(onlyfiles):
        file = onlyfiles[i]
        dat = createProfile(file,onto)
        print(dat)
        final_database = final_database.append(dat)
        i +=1
    
    yield "loop passed success.." + "\n"
    #return final_database
    final_database2 = final_database['Keyword'].groupby([final_database['Candidate Name'], final_database['Subject']]).count().unstack()
    final_database2.reset_index(inplace = True)
    final_database2.fillna(0,inplace=True)
    #print(final_database2)
    yield "Creating Database for candidates..." +"\n"
    excelmat="CandidateSkillExcel"+str(currentDT)+".xlsx"
    writer = ExcelWriter(excelmat)
    final_database2.to_excel(writer,'resume_skill_Sheet')
    writer.save()
    #final_database2 = final_database['Keyword'].groupby([final_database['Candidate Name'], final_database['Subject']]).count().unstack()
    #final_database2.reset_index(inplace = True)
    #final_database2.fillna(0,inplace=True)
    new_data = final_database2.iloc[:,1:]
    new_data.index = final_database2['Candidate Name']
    #execute the below line if you want to see the candidate profile in a csv format
    yield "Generating CSV for candidates..." +"\n"
    csvdata="CandidateSkillcsv"+str(currentDT)+".csv"
    sample2=new_data.to_csv(csvdata)
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size': 10})
    ax = new_data.plot.barh(title="Resume keywords by category", legend=False, figsize=(30,17), stacked=True)
    labels = []
    for j in new_data.columns:
        for i in new_data.index:
            label = str(j)+": " + str(new_data.loc[i][j])
            labels.append(label)
    patches = ax.patches
    for label, rect in zip(labels, patches):
        width = rect.get_width()
        if width > 0:
            x = rect.get_x()
            y = rect.get_y()
            height = rect.get_height()
            ax.text(x + width/2., y + height/2., label, ha='center', va='center')
    mypng="Resume_Statistic"+str(currentDT)+".png"
    plt.savefig(mypng)
    print("Saved sucessfully")
    print("Profile Generated")
    print("Excel Matrix Stored in:" + str(excelmat))
    print("Candidate CSV generated ->" + str(csvdata))
    print("Skill Static Visualization generated -> " + str(excelmat))
    yield "Saved sucessfully" +"\n"
    yield "Profile Generated" +"\n"
    yield "Excel Matrix Stored in:" + str(excelmat) +"\n"
    yield "Candidate CSV generated ->" + str(csvdata) +"\n"
    yield "Skill Visualization generated -> " + str(mypng) +"\n"
    yield "Done" +"\n"
    #return final_database2
    

In [3]:
#mypath="Analytics_hire"
#ontol='datascience_ontology.csv'
#onlyfiles=[os.path.join(mypath,f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,f))]
#rep=get_report(mypath,ontol)
#createProfile(onlyfiles[0],ontol)

In [4]:
#for value in rep:
#    print(value)


In [13]:
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext
from tkinter import Menu
from tkinter import *
from tkinter import messagebox

from tkinter.filedialog import askopenfilename
win=tk.Tk()
win.title("Candidate Resume Parsing")
win.geometry("600x400")
win.resizable(0,0)
scrolw=72
scrolh=19

#scroll
scr=scrolledtext.ScrolledText(win,width=scrolw, height=scrolh, wrap=tk.WORD)
scr.grid(row=5,column=0)

def _quit(): # 7
    win.quit()
    win.destroy()
    exit()

def data1():
    scr.insert(END,"Adding resume folder and Skill Ontology" + "\n")
    scr.insert(END,"Loading...Please Wait" + "\n")
    
    mypath=fldr()
    ttk.Label(labelframe,text=mypath).grid(column=0, row=0,padx=10, pady=10)
    onlyfiles=[os.path.join(mypath,f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,f))]
    print(len(onlyfiles))
    onto=adddict()
    ttk.Label(labelframe,text=onto).grid(column=0, row=1,padx=10, pady=10)
    scr.insert(END,"Generating Candidate Profile.." + " \n")
    d=get_report(mypath,onto)
   
    for values in d:
        scr.insert(END,str(values))
    #print(d)


def adddict():
    folder_selected = askopenfilename()
    fold1=str(folder_selected) 
    #print(fold1)
    #d1=ttk.Label(labelframe,text=fold1).grid(column=0, row=1,padx=10, pady=10)
    return fold1


lab=ttk.Label(win,text="")
lab.grid(column=0, row=0)

def fldr(): 
    folder_selected = filedialog.askdirectory()
    fold=str(folder_selected) 
    print(fold)
    #ttk.Label(labelframe,text=fold).grid(column=0, row=2,padx=10, pady=10)
    return fold

def helpp():
    messagebox.showinfo("Help", """Step 1: Add the candidate resume folder \nStep 2: Add the skill lookup Dictionary \nWait for Exection \nDone """)


#menubar
Menubar=Menu(win)
win.config(menu=Menubar)
filemenu=Menu(Menubar,tearoff=0)
filemenu.add_command(label='Get Report',command=data1)
#filemenu.add_command(label='',command=adddict)
#filemenu.add_command(label='',command=fldr)
filemenu.add_command(label='Exit',command=_quit)
Menubar.add_cascade(label="File",menu=filemenu)
#adding about
filemenu1=Menu(Menubar,tearoff=0)
filemenu1.add_command(label='Help',command=helpp)
Menubar.add_cascade(label="About",menu=filemenu1)

labelframe=ttk.LabelFrame(win,text="File Information")
labelframe.grid(column=0, row=0 ,padx=10, pady=40)

#place label
#lbl1=ttk.Label(labelframe,text="hello").grid(column=0, row=0 ,padx=10, pady=10)
#lbl2=ttk.Label(labelframe,text="world").grid(column=1, row=0,padx=10, pady=10)
#lbl3=ttk.Label(labelframe,text="siraj").grid(column=2, row=0,padx=10, pady=10)

win.mainloop()